# chatbot that can answer user queries from any documents and add a conversational form for collecting user information (Name, Phone Number, Email) when user ask chatbot to call them.

Also, integrate conversational form (book appointment) with tool-agents. Integration of  conversational form with agent-tools, extract complete date format like (YYYY-MM-DD) from users query (eg. Next Monday, and integrate validation in user input with conversational form (like email, phone number)


In [1]:
!pip install -q torch transformers huggingface_hub

In [2]:
!pip install -q transformers[torch]

In [3]:
!pip install -q accelerate

In [9]:
# Import necessary libraries
import torch  # PyTorch for model processing and tensor operations
import transformers  # Hugging Face transformers for model loading and text generation
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline  # Specific classes for loading pre-trained models and tokenizers
from datetime import datetime, timedelta  # For handling date and time calculations
from dateutil import parser  # To parse dates from natural language
import re  # For regular expressions (e.g., validating phone numbers, emails)


In [7]:
# Initialize the chatbot with a pre-trained language model and a list of documents.
class DocumentChatbot:
    def __init__(self, model_name="microsoft/Phi-3-mini-4k-instruct", documents=None):
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="cuda",
            torch_dtype="auto",
            trust_remote_code=True
        )
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.documents = documents or []
        self.user_info = {}
        self.booking_stage = None

    def load_documents(self, documents):
        """Load documents for querying"""
        self.documents = documents

    def extract_date(self, query: str) -> str:
        """Extract date from natural language query, including relative terms."""
        day_mapping = {
            'monday': 0, 'tuesday': 1, 'wednesday': 2, 'thursday': 3,
            'friday': 4, 'saturday': 5, 'sunday': 6
        }

        query_lower = query.lower()

        for day, offset in day_mapping.items():
            if f'next {day}' in query_lower:
                today = datetime.now()
                days_ahead = (offset - today.weekday() + 7) % 7 + 7
                return (today + timedelta(days=days_ahead)).strftime('%Y-%m-%d')

            if day in query_lower:
                today = datetime.now()
                days_ahead = (offset - today.weekday() + 7) % 7
                return (today + timedelta(days=days_ahead)).strftime('%Y-%m-%d')

        if "day after tomorrow" in query_lower:
            return (datetime.now() + timedelta(days=2)).strftime('%Y-%m-%d')
        if "tomorrow" in query_lower:
            return (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
        if "today" in query_lower:
            return datetime.now().strftime('%Y-%m-%d')

        try:
            parsed_date = parser.parse(query, fuzzy=True)
            return parsed_date.strftime('%Y-%m-%d')
        except ValueError:
            return None

    def extract_time(self, query: str) -> str:
        """Extract time from natural language query."""
        try:
            parsed_time = parser.parse(query, fuzzy=True)
            return parsed_time.strftime('%I:%M %p')
        except ValueError:
            return None

    def handle_appointment_booking(self, query: str) -> str:
        """Manage the appointment booking workflow."""
        if not self.booking_stage:
            self.booking_stage = 'name'
            return "Let's book your appointment. What is your full name?"

        if self.booking_stage == 'name':
            self.user_info['name'] = query.strip()
            self.booking_stage = 'phone'
            return "Thank you. What is your phone number? Please provide in the format +1XXXXXXXXXX"

        elif self.booking_stage == 'phone':
            phone_regex = r'^(\+\d{1,3}[- ]?)?\d{10}$'
            if not re.match(phone_regex, query.strip()):
                return "Invalid phone number format. Please enter a 10-digit phone number with optional country code."

            self.user_info['phone_number'] = query.strip()
            self.booking_stage = 'email'
            return "Phone number received. What is your email address?"

        elif self.booking_stage == 'email':
            email_regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
            if not re.match(email_regex, query.strip()):
                return "Invalid email format. Please enter a valid email address."

            self.user_info['email'] = query.strip()
            self.booking_stage = 'date'
            return "Email received. What date would you like to book the appointment? (e.g., next Monday, 2024-06-15)"

        elif self.booking_stage == 'date':
            extracted_date = self.extract_date(query)
            if not extracted_date:
                return "Invalid date format. Please provide a date in YYYY-MM-DD format or use phrases like 'next Monday'."

            self.user_info['appointment_date'] = extracted_date
            self.booking_stage = 'time'
            return f"Date set to {extracted_date}. What time would you like the appointment? (e.g., 10:00 AM or ten o'clock in the morning)"

        elif self.booking_stage == 'time':
            extracted_time = self.extract_time(query)
            if not extracted_time:
                return "Invalid time format. Please use HH:MM AM/PM format (e.g., 10:00 AM)."

            self.user_info['appointment_time'] = extracted_time
            confirmation = f"""
Appointment Details:
Name: {self.user_info['name']}
Phone: {self.user_info['phone_number']}
Email: {self.user_info['email']}
Date: {self.user_info['appointment_date']}
Time: {self.user_info['appointment_time']}

Confirm booking? (yes/no)
"""
            self.booking_stage = 'confirm'
            return confirmation

        elif self.booking_stage == 'confirm':
            if query.lower().strip() in ['yes', 'y']:
                self.booking_stage = None
                return "Appointment booked successfully! We'll send a confirmation to your email."
            else:
                self.booking_stage = None
                self.user_info.clear()
                return "Appointment booking cancelled. How else can I help you?"

        return "I'm not sure how to proceed. Let's start over. Would you like to book an appointment?"

    def query_documents(self, query: str) -> str:
        """Improved semantic document querying."""
        query_lower = query.lower()

        keyword_mappings = {
            'services': ['services', 'offer', 'provide', 'consulting', 'help'],
            'appointment': ['appointment', 'book', 'schedule', 'policy']
        }

        for doc in self.documents:
            doc_lower = doc['content'].lower()
            for keywords in keyword_mappings.values():
                if any(keyword in query_lower for keyword in keywords) and \
                   any(keyword in doc_lower for keyword in keywords):
                    return doc['content']

        return "I couldn't find a matching document. Could you clarify or ask differently?"

    def generate_response(self, query: str) -> str:
        """Generate conversational response."""
        system_prompt = """You are a professional AI assistant for a consulting company.
        Guidelines:
        - Respond warmly and professionally to greetings and casual conversations.
        - For formal queries, provide clear and concise responses.
        - Respond to greetings like "hello" or "hi" naturally with friendly professionalism."""

        full_prompt = f"{system_prompt}\n\nUser Query: {query}"

        pipe = pipeline("text-generation", model=self.model, tokenizer=self.tokenizer)
        generation_args = {"max_new_tokens": 150, "temperature": 0.7, "do_sample": True}

        try:
            response = pipe(full_prompt, **generation_args)[0]['generated_text']
            return response
        except Exception as e:
            return f"I'm having trouble generating a response. Error: {str(e)}"

    def process_query(self, query: str) -> str:

      """Main query processing method."""
      if not query.strip():
          return "I didn't catch that. Could you please repeat?"

      greetings = ["hello", "hi", "hey", "greetings", "good morning", "good evening", "good afternoon"]
      if any(greet in query.lower() for greet in greetings):
          return "Hello! How can I assist you today?"

      # Check for vague queries
      vague_queries = ["how to know it in detail", "tell me more", "explain further", "elaborate"]
      if any(vague in query.lower() for vague in vague_queries):
          return "Could you clarify what you'd like to know more about? For example, services, appointments, or something else?"

      if self.booking_stage is not None:
          return self.handle_appointment_booking(query)

      if any(phrase in query.lower() for phrase in ['book appointment', 'schedule', 'book a time']):
          return self.handle_appointment_booking(query)

      document_result = self.query_documents(query)
      if document_result != "I couldn't find a matching document. Could you clarify or ask differently?":
          return document_result

      return self.generate_response(query)


In [8]:
# Defining sample documents that the chatbot can query.
def main():
    documents = [
        {
            "title": "Services",
            "content": "Our IT consulting services include:\n- Cloud migration\n- Cybersecurity consulting\n- Software development\n- Data Science\n- Digital transformation"
        },
        {
            "title": "Appointments",
            "content": "Appointments are available Monday-Friday, 9 AM - 5 PM. 24-hour cancellation notice required."
        }
    ]

    print("Initializing chatbot...")
    chatbot = DocumentChatbot()
    chatbot.documents = documents

    print("\nWelcome to the IT Consulting Chatbot!")
    print("Type 'exit' to end the conversation.")

    while True:
        user_input = input("\nYou: ").strip()
        if user_input.lower() == 'exit':
            print("Goodbye! Thank you for using our service.")
            break

        response = chatbot.process_query(user_input)
        print("\nChatbot:", response)


if __name__ == "__main__":
    main()

Initializing chatbot...


model-00002-of-00002.safetensors:  66%|######5   | 1.76G/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]


Welcome to the IT Consulting Chatbot!
Type 'exit' to end the conversation.

You: hello

Chatbot: Hello! How can I assist you today?

You: what are your services

Chatbot: Our IT consulting services include:
- Cloud migration
- Cybersecurity consulting
- Software development
- Data Science
- Digital transformation

You: how to know it in detail

Chatbot: Could you clarify what you'd like to know more about? For example, services, appointments, or something else?

You: appointment

Chatbot: Appointments are available Monday-Friday, 9 AM - 5 PM. 24-hour cancellation notice required.

You: so i want to book appointment

Chatbot: Let's book your appointment. What is your full name?

You: abc

Chatbot: Thank you. What is your phone number? Please provide in the format +1XXXXXXXXXX

You: 9807654321

Chatbot: Phone number received. What is your email address?

You: abc@gmail.com

Chatbot: Email received. What date would you like to book the appointment? (e.g., next Monday, 2024-06-15)

You: f